In [ ]:
!pip install pandas scipy tensorflow-hub tensorflow-datasets

In [27]:
from pathlib import Path
from functools import reduce
import pandas as pd

In [28]:
step2_train_path = Path('../datasets/step2_new/header/train')
step2_validation_path = Path('../datasets/step2_new/header/validation')


In [29]:
labels = [path.name for path in step2_train_path.iterdir()]
def prepare_data_frame(step2_path):
    files = {label.name: [file for file in label.iterdir()] for label in step2_path.iterdir()}
    
    file_paths = reduce(lambda x,y: x+y, files.values())
    file_names = list(map(lambda x: x.name, file_paths))
    name_to_path_dict = dict(zip(file_names, map(lambda x: str(x), file_paths)))
    file_path_series = pd.Series(name_to_path_dict, dtype='string', name='file_path')
    
    label_series_list = [
        pd.Series(1.0, index=map(lambda x: x.name, files[label]), name=label)
        for label in labels
    ]
    df = reduce(lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), label_series_list)
    df = df.fillna(value=0.0)
    
    df_with_file_path = pd.merge(df, file_path_series, left_index=True, right_index=True)
    
    return df_with_file_path

In [30]:
train_df = prepare_data_frame(step2_train_path)
validation_df = prepare_data_frame(step2_validation_path)

In [31]:
import tensorflow as tf
import tensorflow_hub as hub

In [32]:
train_ds = tf.data.Dataset.from_tensor_slices({**train_df})
validation_ds = tf.data.Dataset.from_tensor_slices({**validation_df})

In [33]:
def load_image(sample):
    file_path = sample['file_path']
    image_bytes = tf.io.read_file(file_path)
    image = tf.io.decode_image(image_bytes, channels=3, expand_animations=False)
    sample['image'] = image
    return sample

def preprocess_image(sample):
    image = tf.image.convert_image_dtype(sample['image'], tf.float32)
    image = tf.image.resize(image, [224, 224])
    sample['image'] = image
    return sample

def transform_data_for_keras_fit(sample):
    return sample['image'], tuple(sample[label] for label in labels)

In [34]:
trainable_ds = train_ds.map(load_image).map(preprocess_image).map(transform_data_for_keras_fit).batch(32)
testable_ds = validation_ds.map(load_image).map(preprocess_image).map(transform_data_for_keras_fit).batch(32)

In [35]:
inputs = tf.keras.Input(shape=(224, 224, 3), name="image")
efficientnet_v2 = hub.KerasLayer('https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2', trainable=False)
net = efficientnet_v2(inputs)
outputs = [tf.keras.layers.Dense(1, name='dense_'+label)(net) for label in labels]

efficientnet_v2_step2 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [36]:
efficientnet_v2_step2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=[tf.keras.losses.BinaryCrossentropy(from_logits=True, name='binary_crossentropy_'+label) for label in labels],
    metrics=tf.keras.metrics.BinaryAccuracy(),
)

In [38]:
history = efficientnet_v2_step2.fit(
    trainable_ds, epochs=50, validation_data=testable_ds,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/fit')]
)

Epoch 1/50


2023-07-17 05:23:22.494339: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: ../datasets/step2_new/header/train/logo_left/Thumbs.db; No such file or directory
2023-07-17 05:23:22.522045: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 2027284178178185195
2023-07-17 05:23:22.522164: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10234855513871310771
2023-07-17 05:23:22.522341: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8237703765258564134


NotFoundError: Graph execution error:

2 root error(s) found.
  (0) NOT_FOUND:  ../datasets/step2_new/header/train/logo_left/Thumbs.db; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_4]]
  (1) NOT_FOUND:  ../datasets/step2_new/header/train/logo_left/Thumbs.db; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_196182]